In [20]:
import numpy as np
from ssapp.data.AntennaDatasetLoaders import load_serialized_dataset
from ssapp.data.Metrics import relRMSE
from ssapp.Utils import FigureSaver
from torch.utils.data.dataloader import DataLoader
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel,Matern
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from matplotlib import pyplot as plt

%matplotlib inline
%config InlineBackend.figure_formats = ['svg']
%load_ext autoreload
%autoreload 2

figSaver = FigureSaver('GaussianProcessHyperParameterTuning')

dataset_names = ['RFLCT','CircularHornDataset1','PatchAntennaDataset2','MLADataset1']

dataset_name = dataset_names[0]


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
train_dataset = load_serialized_dataset(dataset_names[1]+'_Train')
val_dataset = load_serialized_dataset(dataset_names[1]+'_Val')

train_dataloader = DataLoader(train_dataset,batch_size=len(train_dataset),shuffle=True,num_workers=4)
val_dataloader = DataLoader(val_dataset,batch_size=len(val_dataset),shuffle=True,num_workers=4)

train_params, train_fields = next(iter(train_dataloader))
val_params, val_fields = next(iter(val_dataloader))




In [22]:
print(train_fields.reshape(len(train_fields),-1).shape)

torch.Size([2800, 4332])


In [29]:
from sklearn.neighbors import KNeighborsRegressor as KNNRegressor
# Import sk learn linear regressor
from sklearn.linear_model import LinearRegression as LinearRegression

KNNR = KNNRegressor(n_neighbors=30).fit(train_params,train_fields.reshape(len(train_fields),-1))


pred = KNNR.predict(val_params)
pred_lin = LinReg.predict(val_params)

print(relRMSE(val_fields.flatten(),pred.flatten()))
print(relRMSE(val_fields.flatten(),pred_lin.flatten()))

0.024569714131531736
1.3856389933977857


In [39]:
for dataset_name in dataset_names:
    train_dataset = load_serialized_dataset(dataset_name+'_Train')
    val_dataset = load_serialized_dataset(dataset_name+'_Val')

    train_dataloader = DataLoader(train_dataset,batch_size=len(train_dataset),shuffle=True,num_workers=4)
    val_dataloader = DataLoader(val_dataset,batch_size=len(val_dataset),shuffle=True,num_workers=4)

    train_params, train_fields = next(iter(train_dataloader))
    val_params, val_fields = next(iter(val_dataloader))

    best_remse = np.inf
    best_num_neighbors = 0
    train_loss_at_best_num_neighbors = np.inf


    for num_neigh in range(1,20):
        
        
        KNNR = KNNRegressor(n_neighbors=num_neigh).fit(train_params,train_fields.reshape(len(train_fields),-1))
        pred = KNNR.predict(val_params)
        train_pred = KNNR.predict(train_params)
        remse = relRMSE(val_fields.flatten(),pred.flatten())
        train_remse = relRMSE(train_fields.flatten(),train_pred.flatten())
        if num_neigh == 1:
            print(dataset_name,'KNN Regression at 1 NN',remse,train_remse)
        if remse < best_remse:
            best_remse = remse
            best_num_neighbors = num_neigh
            train_loss_at_best_num_neighbors = train_remse


    print(dataset_name,best_num_neighbors,best_remse)
    print(f'dataset {dataset_name} train loss at best num neighbors {best_num_neighbors} is {train_loss_at_best_num_neighbors}, best remse {best_remse}')

        

ReflectorCutDataset2 KNN Regression at 1 NN 1.5027246096330027 0.0
ReflectorCutDataset2 1 1.5027246096330027
dataset ReflectorCutDataset2 train loss at best num neighbors 1 is 0.0, best remse 1.5027246096330027
CircularHornDataset1 KNN Regression at 1 NN 0.024012983852678405 0.0
CircularHornDataset1 5 0.018118595447877788
dataset CircularHornDataset1 train loss at best num neighbors 5 is 0.01580504536039672, best remse 0.018118595447877788
PatchAntennaDataset2 KNN Regression at 1 NN 0.06885656744999483 0.0
PatchAntennaDataset2 1 0.06885656744999483
dataset PatchAntennaDataset2 train loss at best num neighbors 1 is 0.0, best remse 0.06885656744999483
MLADataset1 KNN Regression at 1 NN 0.9440939244864153 0.0
MLADataset1 2 0.9394742782853145
dataset MLADataset1 train loss at best num neighbors 2 is 0.5243946995245707, best remse 0.9394742782853145


In [52]:
from sklearn.dummy import DummyRegressor
for dataset_name in dataset_names:
    train_dataset = load_serialized_dataset(dataset_name+'_Train')
    val_dataset = load_serialized_dataset(dataset_name+'_Val')

    train_dataloader = DataLoader(train_dataset,batch_size=len(train_dataset),shuffle=True,num_workers=4)
    val_dataloader = DataLoader(val_dataset,batch_size=len(val_dataset),shuffle=True,num_workers=4)

    train_params, train_fields = next(iter(train_dataloader))
    val_params, val_fields = next(iter(val_dataloader))

    DMR = DummyRegressor(strategy='mean')

    DMR.fit(train_params,train_fields.reshape(len(train_fields),-1))

    pred = DMR.predict(val_params)
    train_pred = DMR.predict(train_params)


    remse = relRMSE(val_fields.flatten(),pred.flatten())
    train_remse = relRMSE(train_fields.flatten(),train_pred.flatten())

    print(dataset_name,'Dummy Regression',remse,train_remse)

ReflectorCutDataset2 Dummy Regression 14.975820844124028 14.23430518984844
CircularHornDataset1 Dummy Regression 4.866439283831752 4.824529391236859
PatchAntennaDataset2 Dummy Regression 0.3212554531030823 0.32438049589787926
MLADataset1 Dummy Regression 91.03618168133369 91.78954458662254
